In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [2]:
### Load scalers and encoders

with open("label_encoder_gender.pkl","rb") as file:
    label_encoder_gender = pickle.load(file)

with open("encoder_geography.pkl","rb") as file:
    encoder_geography = pickle.load(file)

with open("standard_scaler.pkl","rb") as file:
    standard_scaler = pickle.load(file)

In [3]:
encoder_geography.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [4]:
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)

In [5]:
### Load the trained ANN model

model = load_model("model.h5")

In [6]:
### Example Input data

input_data = {
    "CreditScore":600,
    "Geography":"France",
    "Gender":"Male",
    "Age":40,
    "Tenure":3,
    "Balance":60000,
    "NumOfProducts":2,
    "HasCrCard":1,
    "IsActiveMember":1,
    "EstimatedSalary":50000
}

In [7]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [8]:
input_df.reset_index(drop=True)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [9]:
### Encode categorical features

### Label encode gender

input_df["Gender"] = label_encoder_gender.transform([input_data["Gender"]])

### One hot encode geography

encoded_geo = encoder_geography.transform([[input_data["Geography"]]]).toarray()
encoded_geo_df = pd.DataFrame(encoded_geo,columns=encoder_geography.get_feature_names_out(["Geography"]))


c:\Users\ViditAgarwal\Desktop\AI ML learned code\Deep Learning\ANN Churn Modelling\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [10]:
encoded_geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
input_df = pd.concat([input_df.drop(["Geography"],axis=1),encoded_geo_df],axis=1)

In [12]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [13]:
### Sacleing the input data

input_scaled = standard_scaler.transform(input_df)
input_scaled

array([[-0.51966032,  0.91025899,  0.10358605, -0.7041574 , -0.2665694 ,
         0.80939072,  0.6464804 ,  0.96680078, -0.86826555,  0.99128795,
        -0.57792762, -0.57003687]])

In [14]:
### Predict Churn

prediction = model.predict(input_scaled)
prediction 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step


array([[0.05251497]], dtype=float32)

In [15]:
probability = prediction[0][0]
probability

np.float32(0.052514967)

In [16]:
if probability > 0.5:
    print("The customer is high likey to churn with probability of {}".format(probability))

else:
    print("The customer less likely to churn, as probabilty to churn being {}".format(probability))

The customer less likely to churn, as probabilty to churn being 0.05251496657729149
